In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [3]:
df = pd.read_csv(f'../data/ATVI/2021-06-01 - 2021-09-01.csv', index_col=0, parse_dates=['time'], dayfirst=True)
df.head()

,time,price
0,2021-06-01 04:00:00+00:00,97.41
1,2021-06-01 04:15:00+00:00,97.76
2,2021-06-01 04:30:00+00:00,97.99
3,2021-06-01 04:45:00+00:00,97.97
4,2021-06-01 05:00:00+00:00,97.80


In [4]:
test_size = 74
train_df = df[:-test_size]
test_df = df[-test_size:]

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape= (x_train.shape[1], 1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))